In [171]:
from itertools import combinations
import numpy as np
from z3 import *
from functools import reduce
import sympy as sp
from sympy import Mul, Pow, Add
x=sp.Symbol('x')
y=sp.Symbol('y')
def antimap(f):
    f = sp.expand(f)
    term=f.as_ordered_terms()
    exp=0
    for i in range(len(term)):
        coef,vari=term[i].as_coeff_Mul()
        exp=exp+coef%2*vari**-1
    return exp
def commute(a,b):
    a = sp.Matrix(a)
    b = sp.Matrix(b)
    lamb=sp.Matrix([[0,0,1,0],
          [0,0,0,1],
          [-1,0,0,0],
          [0,-1,0,0]])
    aa=[]
    for i in range(len(a)):
        aa.append(antimap(a[i]))
    aa=sp.Matrix(aa)
    f=(aa.T*lamb*b)[0]
    f = sp.expand(f)
    term=f.as_ordered_terms()
    exp=0
    for i in range(len(term)):
        coef,vari=term[i].as_coeff_Mul()
        exp=exp+coef%2*vari
    return sp.simplify(exp)
x1=sp.Matrix([1,0,0,0])
x2=sp.Matrix([0,1,0,0])
z1=sp.Matrix([0,0,1,0])
z2=sp.Matrix([0,0,0,1])
basis=[]
basis.append(x1)
basis.append(x2)
basis.append(z1)
basis.append(z2)
basis=sp.Matrix(basis)
def excimap(*A,oper):
    s=[]
    for i,M in enumerate(A):
        s.append(M)
    excitation=[]
    for i in range(len(s)):
        excitation.append(commute(s[i],oper))
    return sp.Matrix(excitation).T
def tdmap(f,m):
    vari=[]
    for j in range(2*m+1):
        for i in range(2*m+1):
            vari.append(x**(i-m)*y**(j-m))
    s=[]
    for i in range(len(vari)):
        s.append(vari[i]*f)
    return sp.Matrix(s)
def m1(*a,m):
    M1=[]
    M1.append(tdmap(excimap(*a,oper=x1),m))
    M1.append(tdmap(excimap(*a,oper=x2),m))
    M1.append(tdmap(excimap(*a,oper=z1),m))
    M1.append(tdmap(excimap(*a,oper=z2),m))
    M1=sp.Matrix(M1)
    return M1

KeyboardInterrupt: 

In [172]:
class F2Poly:
    def __init__(self, terms=None):
        self.terms = {}
        if terms:
            for (i, j), coef in terms.items():
                if coef % 2 == 1:  # 模 2
                    self.terms[(i, j)] = 1  # 只存 1
    def __add__(self, other):
        """F2 上的加法: (f + g) mod 2"""
        result = self.terms.copy()
        for (i, j), coef in other.terms.items():
            if (i, j) in result:
                del result[(i, j)]  # 1+1=0
            else:
                result[(i, j)] = 1
        return F2Poly(result)
    def __mul__(self, other):
        """乘法: f * g"""
        result = {}
        for (i1, j1) in self.terms:
            for (i2, j2) in other.terms:
                i_new, j_new = i1 + i2, j1 + j2
                key = (i_new, j_new)
                if key in result:
                    del result[key]  # 1*1=1, 但可能重复 → 1+1=0
                else:
                    result[key] = 1
        return F2Poly(result)
    def __pow__(self, n):
        if n == 0:
            return F2Poly({(0, 0): 1})  # f^0 = 1

        # 检查 self 是否是单项式（只有一项）或零
        if len(self.terms) == 0:
            if n > 0:
                return F2Poly()  # 0^n = 0 (n>0)
            else:
                raise ValueError("0 的负幂次无定义")
        elif len(self.terms) == 1:
            # 是单项式，如 x^i y^j
            (i, j), coef = next(iter(self.terms.items()))
            # 在 F2 中 coef=1，所以 f = x^i y^j
            if n > 0:
                # 正幂：指数乘 n
                return F2Poly({(i * n, j * n): 1})
            else:
                # 负幂：f^{-n} = x^{-i*n} y^{-j*n}
                return F2Poly({(-i * (-n), -j * (-n)): 1})  # 等价于 (-i*n, -j*n)
        else:
            # 多项式项数 > 1
            if n < 0:
                raise ValueError(f"多项式 {self} 不可逆，无法计算负幂次")
            elif n == 1:
                return self
            elif n % 2 == 0:
                return (self * self) ** (n // 2)
            else:
                return self * (self ** (n - 1))
    def __repr__(self):
        if not self.terms:
            return "0"
        sorted_terms = sorted(self.terms.keys())
        terms = []
        for (i, j) in sorted_terms:
            xi = f"x^{i}" if i != 0 else ""
            yj = f"y^{j}" if j != 0 else ""
            term = xi + yj
            if term == "":
                term = "1"
            terms.append(term)
        return " + ".join(terms)
# 示例 1: 3x + 2y → 模 2 后是 x
p1 = F2Poly({(1, 0): 3, (0, 1): 2,(3,3):5})
p2=F2Poly({(0, 1): 1, (1, 2): 2, (2, 3): 5})
print(p1)
print(p2)
print(p1 **5)

x^1 + x^3y^3
y^1 + x^2y^3
x^5 + x^7y^3 + x^13y^12 + x^15y^15


x: x^1
y: y^1
1/x: x^-1
1/y: y^-1

s1 = 1 + 1/x: x^-1 + 1
s2 = 1 + y: 1 + y^1
s1 * s2: x^-1 + x^-1y^1 + 1 + y^1
f^2 = (1 + x + y)^2: 1 + y^2 + x^2
g*h = (1/x + 1/y)(x + y): x^-1y^1 + x^1y^-1
